In [16]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('SWaT_Dataset_Attack_v1.csv')
#df.shape

# df = df.iloc[:5*len(df.index)//8]
# #df = df.iloc[:len(df.index)//2]
# #df = df.iloc[:7500]
y = df["Normal/Attack"]
del df["Normal/Attack"]
df.head(5)

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,1451296800000000000,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,0.307786,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1
1,1451296801000000000,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,0.307786,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1
2,1451296802000000000,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,0.308619,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1
3,1451296803000000000,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,0.308619,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1
4,1451296804000000000,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,0.308619,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1


In [18]:
y = np.array(y, dtype=str)
y_binary = []
for i in y:
    if i == "Normal":
        y_binary.append(False)  #false == normal
    else:
        y_binary.append(True)   #true == attack
        
y_binary = np.array(y_binary)
y_binary

array([False, False, False, ..., False, False, False])

In [19]:
feature_names = np.array(df.columns, dtype=str)
feature_names = feature_names[1:]

In [20]:
from sklearn.preprocessing import StandardScaler
# Separating out the features
x = df.loc[:, feature_names].values
x_scl = StandardScaler().fit_transform(x)

df_scl = pd.DataFrame(data = x_scl)
df_scl.head(5)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.598055,-0.671759,0.693861,0.66519,-0.083632,1.471045,-1.150243,0.501005,0.656905,0.655351,...,0.296326,0.282974,0.0,0.302690,1.542306,0.313488,-0.102994,0.0,-0.095828,0.0
1,0.614180,-0.671445,0.693861,0.66519,-0.083632,1.471045,-1.150243,0.501005,0.656905,0.655351,...,0.296326,0.282974,0.0,0.302690,1.542306,0.315175,-0.102994,0.0,-0.095828,0.0
2,0.650193,-0.671759,0.693861,0.66519,-0.083632,1.471045,-1.166987,0.501005,0.654184,0.655351,...,0.306616,0.282974,0.0,0.302949,1.542306,0.315175,-0.102994,0.0,-0.095828,0.0
3,0.688087,-0.670818,0.693861,0.66519,-0.083632,1.471045,-1.166987,0.501005,0.654184,0.655351,...,0.306616,0.282974,0.0,0.302949,1.542306,0.313825,-0.102994,0.0,-0.095828,0.0
4,0.717381,-0.666746,0.693861,0.66519,-0.083632,1.471045,-1.166987,0.501005,0.654865,0.655351,...,0.306616,0.282974,0.0,0.302949,1.542306,0.311464,-0.102994,0.0,-0.095828,0.0


In [21]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(df_scl, y_binary, len(feature_names))
print(str(len(cor_feature)), 'selected features')
print(str(len(cor_support)), 'cor_support')

/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


51 selected features
51 cor_support


In [29]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(df_scl)
chi_selector = SelectKBest(chi2, k=len(feature_names))
chi_selector.fit(X_norm, y_binary)
chi_support = chi_selector.get_support()
print(chi_support)
chi_feature = df_scl.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
print(str(len(chi_support)), 'chi_support')

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
51 selected features
51 chi_support


In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), n_features_to_select=len(feature_names), step=10, verbose=5)
rfe_selector.fit(X_norm, y_binary)
rfe_support = rfe_selector.get_support()
rfe_feature = df_scl.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(str(len(rfe_support)), 'rfe_support')

51 selected features
51 rfe_support


In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), max_features=len(feature_names))
embeded_lr_selector.fit(X_norm, y_binary)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = df_scl.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')
print(str(len(embeded_lr_support)), 'embeded_lr_support')

/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


43 selected features
51 embeded_lr_support


In [10]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=len(feature_names))
embeded_rf_selector.fit(df_scl, y_binary)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = df_scl.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')
print(str(len(embeded_rf_support)), 'embeded_rf_support')

21 selected features
51 embeded_rf_support


In [14]:
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support, 'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(len(feature_names))

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,UV401,True,True,True,True,True,5
2,PIT503,True,True,True,True,True,5
3,PIT502,True,True,True,True,True,5
4,PIT501,True,True,True,True,True,5
5,P501,True,True,True,True,True,5
6,P102,True,True,True,True,True,5
7,LIT401,True,True,True,True,True,5
8,LIT301,True,True,True,True,True,5
9,LIT101,True,True,True,True,True,5
10,FIT504,True,True,True,True,True,5


In [15]:
print(feature_selection_df)
selected_features = []
for i in range(1, len(feature_selection_df["Total"])):
    print(feature_selection_df["Feature"][i])
    print(feature_selection_df["Total"][i])
    if feature_selection_df["Total"][i] > 3:
        selected_features.append(feature_selection_df["Feature"][i])

print(selected_features)

    Feature  Pearson  Chi-2   RFE  Logistics  Random Forest  Total
1     UV401     True   True  True       True           True      5
2    PIT503     True   True  True       True           True      5
3    PIT502     True   True  True       True           True      5
4    PIT501     True   True  True       True           True      5
5      P501     True   True  True       True           True      5
6      P102     True   True  True       True           True      5
7    LIT401     True   True  True       True           True      5
8    LIT301     True   True  True       True           True      5
9    LIT101     True   True  True       True           True      5
10   FIT504     True   True  True       True           True      5
11   FIT503     True   True  True       True           True      5
12   FIT501     True   True  True       True           True      5
13   FIT401     True   True  True       True           True      5
14   FIT201     True   True  True       True           True   

In [13]:
df_fs = df[selected_features]
df_fs.head()

""
0
1
2
3
4
